# Simulating Language, Lab 8, Convergence to the Prior

This simulation implements a simplified version of the language model from Kirby, Dowman & Griffiths (2007) using an explicit agent-based simulation.

In [ ]:
import random
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf')

from math import log, log1p, exp
from scipy.special import logsumexp

from numpy import mean # This is a handy function that calculate the average of a list

Following Kirby, Dowman & Griffiths (2007), we assume a language is made up of a set of *variables*, each of which can exist in a number of different *variant* forms. This is a rather general characterisation that actually applies well to a number of linguistic phenomena. For example, we can think of the variables as different syntactic categories, and the variants as word orders. Alternatively, the variables could be verb-meanings and the variants different realisations of the past tense, and so on. Agents will produce (and learn from) data which simply exemplifies which variant they have for a particular variable (with the possibility of noise on transmission). We will group languages into two classes: regular languages (where the same variant is used for all variables) and irregular languages (where more than one variant is used).

In [ ]:
variables = 2           # The number of different variables in the language
variants = 2            # The number of different variants each variable can take

## Functions for dealing with log probabilities

Here are our standard functions for dealing with logs, as before.

In [ ]:
def log_subtract(x,y):
    return x + log1p(-exp(y - x))

def normalize_logprobs(logprobs):
    logtotal = logsumexp(logprobs) #calculates the summed log probabilities
    normedlogs = []
    for logp in logprobs:
        normedlogs.append(logp - logtotal) #normalise - subtracting in the log domain
                                        #equivalent to dividing in the normal domain
    return normedlogs
 
def log_roulette_wheel(normedlogs):
    r = log(random.random()) #generate a random number in [0,1), then convert to log
    accumulator = normedlogs[0]
    for i in range(len(normedlogs)):
        if r < accumulator:
            return i
        accumulator = logsumexp([accumulator, normedlogs[i + 1]])

We're also going to have a function that works a little like the `log_roulette_wheel` but instead always picks the most probable index (instead of picking indices proportional to their probability). We sometimes call this a "winner take all" function. While `log_roulette_wheel` can be used to implement *sampling*, `wta` can be used to implement *MAP* hypothesis selection. If there is more than one winner, then we pick randomly among them.

In [ ]:
def wta(probs):
    maxprob = max(probs) # Find the maximum probability (works if these are logs or not)
    candidates = []
    for i in range(len(probs)):
        if probs[i] == maxprob:
            candidates.append(i) # Make a list of all the indices with that maximum probability
    return random.choice(candidates)

## Production of data

In [ ]:
def produce(language, log_error_probability):
    variable = random.randrange(len(language)) # Pick a variant to produce
    correct_variant = language[variable]
    if log(random.random()) > log_error_probability:
        return variable, correct_variant # Return the variable, variant pair
    else:
        possible_error_variants = list(range(variants))
        possible_error_variants.remove(correct_variant)
        error_variant = random.choice(possible_error_variants)
        return variable, error_variant

The function produce takes a language, selects a random variable, and produces the relevant variant from the language, with a certain probability of error.

- By looking at this code, can you tell how languages are represented in the simulation?
- Can you see how errors on production work?

## Classifying languages

In this language model, prior probability is determined by language class: regular languages differ from irregular languages in their prior probability, and ultimately we are interested in the proportion of our simulated population who use regular languages. We therefore need a function to take a language and classify it as regular or not - the function `regular` does this.

At this point, I should highlight a bit of a problematic bit of terminology that is used in the field. We've already seen the word "regular" in this course with our beta-binomial simulations, referring to whether or not something is given a variable label or a regular one. We're not using "regular" in that sense here. Instead we're using it in the same way linguists refer to "regular verbs" or "irregular verbs", for example. Regular verbs in English are ones that use a general strategy for forming the past tense (for example: *walk/walked*) whereas irregular verbs are ones that use an idiosyncratic strategy for forming the past tense (for example: *go/went*). Our language is "regular" in this sense if it uses the same strategy for each of its meanings. 

In [ ]:
def regular(language):
    first_variant = language[0]
    for variant in language:
        if variant != first_variant:
            return False # The language can only be regular if every variant is the same as the first
    return True

## The Bayesian bits

In [ ]:
def logprior(language, log_bias):
    if regular(language):
        number_of_regular_languages = variants
        return log_bias - log(number_of_regular_languages) #subtracting logs = dividing
    else:
        number_of_irregular_languages = variants ** variables - variants # the double star here means raise to the power
                                                                         # e.g. 4 ** 2 is four squared
        return log_subtract(0, log_bias) - log(number_of_irregular_languages)
        # log(1) is 0, so log_subtract(0, bias) is equivalent to (1 - bias) in the
        # non-log domain

The function `logprior` returns the prior probability (as a log probability) of a particular language. The strength of preference for regular languages is given as the second argument - if bias is over 0.5 (when converted back from a log probability), regular languages have higher prior probability.

- Why are we dividing the bias by the number of regular and irregular languages in this function? Check you understand how these numbers are calculated.
- How does this function differ from the prior from the Kirby, Dowman & Griffiths (2007) paper? (Hint: consider the case of more than two variables.)

In [ ]:
def loglikelihood(data, language, log_error_probability):
    loglikelihoods = []
    logp_correct = log_subtract(0, log_error_probability) #probability of producing correct form
    logp_incorrect = log_error_probability - log(variants - 1) #logprob of each incorrect variant
    for utterance in data:
        variable = utterance[0]
        variant = utterance[1]
        if variant == language[variable]:
            loglikelihoods.append(logp_correct)
        else:
            loglikelihoods.append(logp_incorrect)
    return sum(loglikelihoods) #summing log likelihoods = multiplying likelihoods

The function `loglikelihood` takes a language and a list of data and works out the (log) likelihood of the data given the language. We allows some small probability (given by the third argument) that a speaker will produce the ‘wrong’ variant, i.e. a variant other than that specified by their language.

## Learning

Bayesian learners calculate the posterior probability of each language based on some data, then select a language (‘learn’) based on those posterior probabilities. `learn` implements this. As discussed in the lecture, there are two ways you could select a language based on the posterior probability distribution:
- You could pick the best language - i.e. the language with the highest posterior probability. This is called MAP (“maximum a posteriori”) learning.
- Alternatively, you could pick a language probabilistically based on its posterior probability, without necessarily going for the best one every time (e.g. if language 0 has twice the posterior probability of language 1, you are twice as likely to pick it). This is called sampling (for “sampling from the posterior distribution”).

The next bits of code implements both these ways of learning, using the `wta` function to do MAP learning and using `log_roulette_wheel` to do sampling (from previous labs, which assumed learners sample from the posterior). `all_languages` enumerates all possible languages for expressing `variables` variables and `variants` variants using a cute recursive method.

In [ ]:
def all_languages(variables, variants):
    if variables == 0:
        return [[]] # The list of all languages with zero variables is just one language, and that's empty
    else:
        result = [] # If we are looking for a list of languages with more than zero variables, 
                    # then we'll need to build a list
        smaller_langs = all_languages(variables - 1, variants) # Let's first find all the languages with one 
                                                               # fewer variables
        for language in smaller_langs: # For each of these smaller languages, we're going to have to create a more
                                       # complex language by adding each of the possible variants
            for variant in range(variants):
                result.append(language + [variant])
        return result

Don’t worry too much if you can’t figure out how it works, but you might get an idea if you figure out what steps it would take when called with different arguments, like `all_languages(0, 2)`, `all_languages(1, 2)`, `all_languages(2, 2)`, `all_languages(0, 3)`, `all_languages(1, 3)` and so on. Try that out in the cell below. You should also now be able to see if you were right with your answer to the question above as to how languages are represented in this code.

Finally, `learn` implements hypothesis selection.

In [ ]:
def learn(data, log_bias, log_error_probability, learning_type):
    list_of_all_languages = all_languages(variables, variants) # uses the parameters we set above
    list_of_posteriors = []
    for language in list_of_all_languages:
        this_language_posterior = loglikelihood(data, language, log_error_probability) + logprior(language, log_bias)
        list_of_posteriors.append(this_language_posterior)
    if learning_type == 'map':
        map_language_index = wta(list_of_posteriors) # For MAP learning, we pick the best language
        map_language = list_of_all_languages[map_language_index]
        return map_language
    if learning_type == 'sample':
        normalized_posteriors = normalize_logprobs(list_of_posteriors)
        sampled_language_index = log_roulette_wheel(normalized_posteriors) # For sampling, we use the roulette wheel
        sampled_language = list_of_all_languages[sampled_language_index]
        return sampled_language

## Iterated learning

`iterate`, is the top-level function which actually runs the simulation. It starts with a random language from the set of possible languages, and then each generation a learner learns from data produced by the previous agent. This function returns a list indicating whether the language is regular or not each generation. For convenience we encode the regular languages as `1` in this list and `0` otherwise. It also returns a second list, showing what each language was per generation in case you want more details. (Generally, we're just going to be using the first list for plotting graphs etc.)

In [ ]:
def iterate(generations, bottleneck, log_bias, log_error_probability, learning_type):
    language = random.choice(all_languages(variables, variants))
    if regular(language):
        accumulator = [1]
    else:
        accumulator = [0]
    language_accumulator = [language]
    for generation in range(generations):
        data = []
        for i in range(bottleneck):
            data.append(produce(language, log_error_probability))
        language = learn(data, log_bias, log_error_probability, learning_type)
        if regular(language):
            accumulator.append(1)
        else:
            accumulator.append(0)
        language_accumulator.append(language)
    return accumulator, language_accumulator

So, for example, to do an iterated learning simulation for 100 generations, a bottleneck of 5, with a bias in favour of regularity of 0.6, an error probability of 0.05, with sampling as the learning type, you'd use:

```
iterate(1000, 5, log(0.6), log(0.05), 'sample')
```

## Questions

**Note:** To answer some of these questions, you're going to have to run quite long simulation runs (up to 100000 generations to get super accurate results). In general, you probably want to keep the bottleneck values between 1 and 10.

1. Using the default parameters suggested above, can you demonstrate convergence to the prior? You may want to start with a line graph of the results to get a sense of what's going on in the simulation. You could also plot a histogram to get the overall proportion of regular languages in the whole run, but notice that this is just the same as the average of your whole list so you don't really need a graph for this! You can get the average directly by using, for example, `print(mean(data[0]))` if `data` is the result of your simulation run (remember the `iterate` function returns whether the language at the each generation is regular or not as the first part of the data it returns, which is why we use `data[0]`.

2. How does changing the error rate and bottleneck size affect the results for the sample learner? Make sure you run the simulation long enough to get repeatable results.

3. Switch the learning type to MAP learning, and rerun the simulation. Can you show the amplification of prior bias that is shown in the paper?

4. How is bias amplification affected by the bottleneck size? Can you run a range of simulations for different bottleneck sizes, find the means, and plot these nicely in a graph?